### Preparations for work with caggle

(Worked for google collab only)

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
# # In order to proceed this, one need go to https://www.kaggle.com/me/account and select Create API Token.
# # This will trigger the download of kaggle.json. Then it should be uploaded in collab files
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

### Downloading dataset I am interested in

In [ ]:
! kaggle datasets download saroz014/plant-diseases/dataset_itr2

In [ ]:
! unzip plant-diseases.zip

### Copying only folders with apples we are interested in this research to specific folder

(The easiest way for me how to do that)

In [ ]:
from distutils.dir_util import copy_tree
copy_tree("archive/dataset_itr2/dataset_itr2/train/Apple___Apple_scab", "main_set/train/Apple___Apple_scab")
copy_tree("archive/dataset_itr2/dataset_itr2/train/Apple___Black_rot", "main_set/train/Apple___Black_rot")
copy_tree("archive/dataset_itr2/dataset_itr2/train/Apple___Cedar_apple_rust", "main_set/train/Apple___Cedar_apple_rust")
copy_tree("archive/dataset_itr2/dataset_itr2/train/Apple___healthy", "main_set/train/Apple___healthy")

copy_tree("archive/dataset_itr2/dataset_itr2/test/Apple___Apple_scab", "main_set/test/Apple___Apple_scab")
copy_tree("archive/dataset_itr2/dataset_itr2/test/Apple___Black_rot", "main_set/test/Apple___Black_rot")
copy_tree("archive/dataset_itr2/dataset_itr2/test/Apple___Cedar_apple_rust", "main_set/test/Apple___Cedar_apple_rust")
copy_tree("archive/dataset_itr2/dataset_itr2/test/Apple___healthy", "main_set/test/Apple___healthy")


### Count number of files here

(next two cells have sense only if folder main_set have no 'validation folder'. They change dataset from train/test 80/20 to train/validation/test 65/15/20)

In [ ]:
import os
list_names = ['Apple___Apple_scab', 'Apple___Black_rot',
              'Apple___Cedar_apple_rust', 'Apple___healthy']

dict_amounts = dict()

for folder in ['test', 'train']:
    for subfolder in list_names:
        cur_path = 'main_set/{}/{}'.format(folder, subfolder)
        num_files = len(os.listdir(cur_path))
        dict_amounts[cur_path] = num_files
        
dict_amounts

# if validation folder is present, these will be 0.3, else 0.25
for subfolder in list_names:
    print(dict_amounts['main_set/test/{}'.format(subfolder)]/dict_amounts['main_set/train/{}'.format(subfolder)])

In [ ]:
import shutil
import os

for subfolder in list_names:
    val_list = os.listdir('main_set/train/{}'.format(subfolder))
    val_list.sort()
    # 0.1875 of train set will be 0.15 of all dataset
    index = int(len(val_list) * 0.1875)
    # should take number that is divided by 4 because in dataset there are 4 photos for each leaf, just spinned
    val_list = val_list[:index - index%4]

    os.makedirs('main_set/validation/{}/'.format(subfolder))
    for g in val_list:  
        os.rename('main_set/train/{}/{}'.format(subfolder, g), 'main_set/validation/{}/{}'.format(subfolder, g))


In [ ]:
# if one want to delete all (unused) dataset
# os.remove('plant-diseases.zip')
# shutil.rmtree('dataset_itr2')

## Work with torchvision models


In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
data_dir = "main_set"

num_classes = 4

batch_size = 8

num_epochs = 10

feature_extract = False

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)


                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'validation':
                val_acc_history.append(epoch_acc)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best validation Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "densenet":
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


In [ ]:
def transformations(input_size):
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'validation': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    print("Initializing Datasets and Dataloaders...")


    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test', 'validation']}

    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'test', 'validation']}

    print("Finished.")
    
    return dataloaders_dict

### Training models

In [ ]:
dict_res = dict()

for model_name in ['densenet','vgg', 'resnet', 'alexnet']:
    model_ft, input_size = initialize_model(model_name, num_classes, False, use_pretrained=True)

    print(model_ft)
    model_ft = model_ft.to(device)

    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    dataloaders_dict = transformations(input_size)

    model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

    dict_res[model_name] = hist
    # To save models
#     file_name = 'main_set/model_{}.pt'.format(model_name)
#     torch.save(model_ft.state_dict(), file_name)
    